# 🚀 ML Trading System - Interaktívna Explorácia Dát

Tento notebook ti ukáže **krok-po-kroku** čo pipeline robí.

Môžeš si:
- Pozrieť každý krok pipeline
- Vidieť grafy a vizualizácie
- Pohrať sa s dátami
- Experimentovať s rôznymi tickermi

---

## 📦 Import knižníc

In [ ]:
# 🔧 FIX: Pridaj adresár projektu do Python path
import sys
import os

# ABSOLÚTNA cesta k projektovému adresáru
project_dir = r'C:\Users\milan\Desktop\Git-Projects\ml_trading_system'

# Pridaj do Python path
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

# Zmeň working directory na projektový adresár
os.chdir(project_dir)

print(f"✅ Python path nastavený: {project_dir}")
print(f"✅ Working directory: {os.getcwd()}")
print(f"✅ Moduly data_collector, feature_engineering, data_validator sú teraz dostupné!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Naše moduly
from data_collector import DataCollector
from feature_engineering import FeatureEngineer
from data_validator import generate_data_quality_report

# Nastavenia pre grafy
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Všetky knižnice načítané!")

---

## 🎯 KROK 1: Stiahnutie Dát

Stiahneme historické dáta pre **SPY** (S&P 500 ETF) z Yahoo Finance.

**Môžeš zmeniť:**
- `TICKER` na iný ticker (napr. 'QQQ', 'AAPL', 'TSLA')
- `START_DATE` a `END_DATE` na iné obdobie

In [ ]:
# ⚙️ KONFIGURÁCIA - MÔŽEŠ ZMENIŤ!
TICKER = 'SPY'
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'

print(f"📊 Sťahujem dáta pre {TICKER} od {START_DATE} do {END_DATE}...\n")

# Vytvoríme DataCollector
collector = DataCollector(TICKER)

# Stiahnutie dát
df_raw = collector.download_historical(START_DATE, END_DATE)

print(f"\n✅ Stiahnuté: {len(df_raw)} riadkov")
print(f"📅 Od: {df_raw.index[0].strftime('%Y-%m-%d')}")
print(f"📅 Do: {df_raw.index[-1].strftime('%Y-%m-%d')}")

### 👀 Pozrime sa na surové dáta

In [ ]:
# Prvých 10 riadkov
print("📋 Prvých 10 riadkov:")
df_raw.head(10)

In [ ]:
# Základné štatistiky
print("📊 Základné štatistiky:")
df_raw.describe()

### 📈 Graf 1: Vývoj Ceny (Close Price)

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_raw.index, df_raw['Close'], linewidth=2, label='Close Price')
plt.title(f'{TICKER} - Vývoj Ceny (2020-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Cena ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Vidíš COVID crash (marec 2020) a následný rast!")

### 📊 Graf 2: Volume (Objem obchodovania)

In [ ]:
plt.figure(figsize=(14, 5))
plt.bar(df_raw.index, df_raw['Volume'], width=1.0, alpha=0.7, label='Volume')
plt.title(f'{TICKER} - Objem Obchodovania', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Volume', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Vysoký volume počas COVID crash = veľa paniky!")

---

## 🔧 KROK 2: Vytvorenie Features (príznakov)

Teraz vytvoríme **11 technických indikátorov**:

### Returns (výnosy):
- `return_1d` - 1-dňový výnos
- `return_5d` - 5-dňový výnos
- `return_10d` - 10-dňový výnos
- `return_20d` - 20-dňový výnos

### Volatility (volatilita):
- `volatility_20d` - 20-dňová volatilita
- `volatility_60d` - 60-dňová volatilita

### Volume:
- `volume_ratio` - pomer voči 20-dňovému priemeru

### Price Position:
- `price_position` - pozícia ceny v 20-dňovom rozpätí (0-1)

### Trend:
- `sma_20` - 20-dňový kĺzavý priemer
- `sma_50` - 50-dňový kĺzavý priemer
- `trend` - 1 ak sma_20 > sma_50 (uptrend)

In [ ]:
print("🔧 Vytváram features...\n")

engineer = FeatureEngineer()
df_featured = engineer.create_basic_features(df_raw.copy())

print(f"✅ Vytvorených {len(engineer.get_feature_names())} features!")
print(f"\n📋 Features:")
for i, feat in enumerate(engineer.get_feature_names(), 1):
    print(f"  {i}. {feat}")

### 👀 Pozrime sa na features

In [ ]:
# Ukáž prvých 100 riadkov (aby sme videli aj non-NaN hodnoty)
print("📋 Dáta s features (riadky 60-70, aby sme videli platné hodnoty):")
df_featured[['Close', 'return_1d', 'return_5d', 'volatility_20d', 'volume_ratio', 'trend']].iloc[60:70]

### 📊 Graf 3: Returns (výnosy)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1-day returns
axes[0, 0].plot(df_featured.index, df_featured['return_1d'], linewidth=1, alpha=0.7)
axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0, 0].set_title('1-Day Returns', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Return', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

# 5-day returns
axes[0, 1].plot(df_featured.index, df_featured['return_5d'], linewidth=1, alpha=0.7)
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0, 1].set_title('5-Day Returns', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Return', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

# 10-day returns
axes[1, 0].plot(df_featured.index, df_featured['return_10d'], linewidth=1, alpha=0.7)
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_title('10-Day Returns', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Return', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)

# 20-day returns
axes[1, 1].plot(df_featured.index, df_featured['return_20d'], linewidth=1, alpha=0.7)
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 1].set_title('20-Day Returns', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Return', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Čím dlhšie obdobie, tým väčšie výkyvy (ale menej častejšie zmeny)")

### 📊 Graf 4: Volatilita

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_featured.index, df_featured['volatility_20d'], label='20-day Volatility', linewidth=2)
plt.plot(df_featured.index, df_featured['volatility_60d'], label='60-day Volatility', linewidth=2)
plt.title('Volatilita (Štandardná odchýlka výnosov)', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Volatilita', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Vysoká volatilita v marci 2020 (COVID) a začiatkom 2022 (Bear market)")

### 📊 Graf 5: Trend Indikátory (Moving Averages)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(df_featured.index, df_featured['Close'], label='Close Price', linewidth=2, alpha=0.7)
plt.plot(df_featured.index, df_featured['sma_20'], label='SMA 20', linewidth=2, linestyle='--')
plt.plot(df_featured.index, df_featured['sma_50'], label='SMA 50', linewidth=2, linestyle='--')
plt.title('Cena a Kĺzavé Priemery (Moving Averages)', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Cena ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Keď SMA20 > SMA50 = UPTREND (trend = 1)")
print("💡 Keď SMA20 < SMA50 = DOWNTREND (trend = 0)")

### 📊 Graf 6: Price Position (Pozícia v rozpätí)

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df_featured.index, df_featured['price_position'], linewidth=2)
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Stred (0.5)')
plt.axhline(y=0.8, color='orange', linestyle='--', alpha=0.5, label='Blízko high (0.8)')
plt.axhline(y=0.2, color='green', linestyle='--', alpha=0.5, label='Blízko low (0.2)')
plt.title('Price Position (Pozícia ceny v 20-dňovom rozpätí)', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Position (0 = low, 1 = high)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Price position = 1 → cena je na 20-dňovom maxime")
print("💡 Price position = 0 → cena je na 20-dňovom minime")

---

## 🎯 KROK 3: Vytvorenie Target Variable (Cieľová premenná)

**Target** = 5-dňový výnos do budúcnosti

⚠️ **CRITICAL:** Target je správne posunutý dopredu = **NO LOOK-AHEAD BIAS!**

V čase `t` predikujeme výnos od `t` do `t+5`.

In [ ]:
HOLDING_PERIOD = 5

print(f"🎯 Vytváram target variable: {HOLDING_PERIOD}-dňový forward return...\n")

df_featured = engineer.create_target(df_featured, horizon=HOLDING_PERIOD)

print(f"✅ Target vytvorený!")
print(f"\n📊 Target statistics:")
print(df_featured['target'].describe())

### 🔍 Overenie: Target je naozaj forward return?

In [ ]:
# Vezmime riadok 100 a overíme target manuálne
test_idx = 100

# Target z modelu
target_model = df_featured['target'].iloc[test_idx]

# Manuálny výpočet: (Close[t+5] / Close[t]) - 1
close_t = df_featured['Close'].iloc[test_idx]
close_t5 = df_featured['Close'].iloc[test_idx + 5]
target_manual = (close_t5 / close_t) - 1

print(f"🔍 Overenie na riadku {test_idx}:")
print(f"\n  Close[t={test_idx}]:     ${close_t:.2f}")
print(f"  Close[t+5={test_idx+5}]: ${close_t5:.2f}")
print(f"\n  Target (model):  {target_model:.6f}")
print(f"  Target (manuál): {target_manual:.6f}")
print(f"\n  ✅ Zhoduje sa: {np.isclose(target_model, target_manual)}")

print("\n💡 Target je správne! Predikujeme BUDÚCI výnos, nie minulý!")

### 📊 Graf 7: Distribúcia Target Variable

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogram
axes[0].hist(df_featured['target'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero return')
axes[0].set_title('Distribúcia 5-Day Forward Returns', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Return', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend(fontsize=12)
axes[0].grid(True, alpha=0.3)

# Time series
axes[1].plot(df_featured.index, df_featured['target'], linewidth=1, alpha=0.7)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('5-Day Forward Returns Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Dátum', fontsize=12)
axes[1].set_ylabel('Target Return', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

positive_pct = (df_featured['target'].dropna() > 0).sum() / len(df_featured['target'].dropna()) * 100
print(f"💡 {positive_pct:.1f}% období má pozitívny 5-dňový výnos")

---

## 🧹 KROK 4: Čistenie Dát (Odstránenie NaN)

Prvé riadky majú NaN hodnoty kvôli rolling window features.

Napríklad:
- `volatility_20d` potrebuje 20 dní histórie
- `volatility_60d` potrebuje 60 dní histórie
- `target` má NaN na posledných 5 riadkoch (nemáme budúce dáta)

In [ ]:
print("🧹 Čistenie dát (odstránenie NaN)...\n")

rows_before = len(df_featured)
print(f"  Riadkov pred čistením: {rows_before}")

# Odstránime NaN v features a target
feature_cols = engineer.get_feature_names()
cols_to_check = feature_cols + ['target']
df_clean = df_featured.dropna(subset=cols_to_check)

rows_after = len(df_clean)
rows_lost = rows_before - rows_after

print(f"  Riadkov po čistení:    {rows_after}")
print(f"  Stratených:            {rows_lost} ({rows_lost/rows_before*100:.1f}%)")
print(f"\n✅ Dáta sú čisté!")

---

## ✂️ KROK 5: Train/Test Split (Temporálne delenie)

**DÔLEŽITÉ:** V time-series **NIKDY** nešaflujem dáta!

Train set = Prvých 70% (chronologicky)
Test set = Posledných 30% (chronologicky)

⚠️ **NO SHUFFLING** - zachovávame časovú postupnosť!

In [ ]:
TRAIN_SPLIT = 0.7

print(f"✂️ Delím dáta na train/test ({int(TRAIN_SPLIT*100)}/{int((1-TRAIN_SPLIT)*100)})...\n")

# Temporálny split (BEZ shufflovania!)
split_idx = int(len(df_clean) * TRAIN_SPLIT)

train_df = df_clean.iloc[:split_idx].copy()
test_df = df_clean.iloc[split_idx:].copy()

print(f"📊 Train set:")
print(f"  - Riadkov: {len(train_df)} ({len(train_df)/len(df_clean)*100:.1f}%)")
print(f"  - Od: {train_df.index[0].strftime('%Y-%m-%d')}")
print(f"  - Do: {train_df.index[-1].strftime('%Y-%m-%d')}")

print(f"\n📊 Test set:")
print(f"  - Riadkov: {len(test_df)} ({len(test_df)/len(df_clean)*100:.1f}%)")
print(f"  - Od: {test_df.index[0].strftime('%Y-%m-%d')}")
print(f"  - Do: {test_df.index[-1].strftime('%Y-%m-%d')}")

# Overenie, že sa neprekrývajú
if train_df.index[-1] < test_df.index[0]:
    print(f"\n✅ Train a test sa neprekrývajú!")
else:
    print(f"\n❌ CHYBA: Train a test sa prekrývajú!")

### 📊 Graf 8: Train/Test Split Vizualizácia

In [ ]:
plt.figure(figsize=(14, 6))

# Train data
plt.plot(train_df.index, train_df['Close'], label='Train Data', color='blue', linewidth=2)

# Test data
plt.plot(test_df.index, test_df['Close'], label='Test Data', color='orange', linewidth=2)

# Split line
plt.axvline(x=train_df.index[-1], color='red', linestyle='--', linewidth=2, label='Train/Test Split')

plt.title('Train/Test Split (Temporálne delenie)', fontsize=16, fontweight='bold')
plt.xlabel('Dátum', fontsize=12)
plt.ylabel('Close Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Modrá = dáta na trénovanie modelu")
print("💡 Oranžová = dáta na testovanie modelu (nevidené!)")

---

## 📊 KROK 6: Analýza Features (Korelácia)

Pozrime sa, ktoré features najviac korelujú s targetom.

In [ ]:
# Vyberieme len features a target pre train set
train_features = train_df[feature_cols + ['target']]

# Vypočítame koreláciu s targetom
correlations = train_features.corr()['target'].drop('target').sort_values(ascending=False)

print("📊 Korelácia features s targetom (5-day return):")
print("\n" + "="*50)
for feat, corr in correlations.items():
    bar = '█' * int(abs(corr) * 50)
    sign = '+' if corr > 0 else '-'
    print(f"{feat:20} {sign} {bar} {corr:.4f}")
print("="*50)

print("\n💡 Čím väčší absolútny korelačný koeficient, tým silnejší vzťah!")

### 📊 Graf 9: Correlation Heatmap

In [ ]:
# Correlation matrix
corr_matrix = train_features.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 Červená = pozitívna korelácia, Modrá = negatívna korelácia")

---

## 🎮 KROK 7: Experimentuj s Dátami!

Teraz si môžeš pohrať s dátami. Tu je niekoľko nápadov:

### 💡 Nápady na experimenty:

1. **Zmeň ticker** - Skús QQQ, AAPL, TSLA
2. **Zmeň holding period** - Skús 10-dňový alebo 20-dňový target
3. **Porovnaj volatility** - V ktorých obdobiach bola najvyššia?
4. **Analyzuj trendy** - Koľko percent času je trh v uptrende?
5. **Nájdi najlepší deň** - Ktorý deň mal najvyšší 5-day return?

### 🎯 Experiment 1: Najlepšie a najhoršie obdobia

In [ ]:
# Top 10 najlepších 5-day returns
print("🚀 TOP 10 Najlepších 5-day returns:")
print("="*60)
top_returns = df_clean.nlargest(10, 'target')[['Close', 'target', 'volatility_20d', 'trend']]
print(top_returns)

print("\n💣 TOP 10 Najhorších 5-day returns:")
print("="*60)
worst_returns = df_clean.nsmallest(10, 'target')[['Close', 'target', 'volatility_20d', 'trend']]
print(worst_returns)

### 🎯 Experiment 2: Uptrend vs Downtrend

In [ ]:
# Rozdelíme dáta podľa trendu
uptrend = df_clean[df_clean['trend'] == 1]
downtrend = df_clean[df_clean['trend'] == 0]

print("📊 Analýza Uptrend vs Downtrend:")
print("\n" + "="*60)
print(f"\n🟢 UPTREND (SMA20 > SMA50):")
print(f"  - Počet dní: {len(uptrend)} ({len(uptrend)/len(df_clean)*100:.1f}%)")
print(f"  - Priemerný 5-day return: {uptrend['target'].mean():.4f} ({uptrend['target'].mean()*100:.2f}%)")
print(f"  - Štandardná odchýlka: {uptrend['target'].std():.4f}")

print(f"\n🔴 DOWNTREND (SMA20 < SMA50):")
print(f"  - Počet dní: {len(downtrend)} ({len(downtrend)/len(df_clean)*100:.1f}%)")
print(f"  - Priemerný 5-day return: {downtrend['target'].mean():.4f} ({downtrend['target'].mean()*100:.2f}%)")
print(f"  - Štandardná odchýlka: {downtrend['target'].std():.4f}")
print("="*60)

# Vizualizácia
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogram pre uptrend
axes[0].hist(uptrend['target'], bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0].axvline(x=uptrend['target'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].set_title('Uptrend: 5-Day Returns', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Return', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Histogram pre downtrend
axes[1].hist(downtrend['target'], bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1].axvline(x=downtrend['target'].mean(), color='blue', linestyle='--', linewidth=2, label='Mean')
axes[1].set_title('Downtrend: 5-Day Returns', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Return', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 V uptrende sú vyššie výnosy? Alebo naopak?")

### 🎯 Experiment 3: Vysoká vs Nízka Volatilita

In [ ]:
# Rozdelíme podľa volatility (nad/pod mediánom)
median_vol = df_clean['volatility_20d'].median()

high_vol = df_clean[df_clean['volatility_20d'] > median_vol]
low_vol = df_clean[df_clean['volatility_20d'] <= median_vol]

print(f"📊 Analýza Vysoká vs Nízka Volatilita (medián = {median_vol:.4f}):")
print("\n" + "="*60)
print(f"\n⚡ VYSOKÁ VOLATILITA:")
print(f"  - Počet dní: {len(high_vol)}")
print(f"  - Priemerný 5-day return: {high_vol['target'].mean():.4f} ({high_vol['target'].mean()*100:.2f}%)")
print(f"  - Štandardná odchýlka: {high_vol['target'].std():.4f}")
print(f"  - Max return: {high_vol['target'].max():.4f}")
print(f"  - Min return: {high_vol['target'].min():.4f}")

print(f"\n😌 NÍZKA VOLATILITA:")
print(f"  - Počet dní: {len(low_vol)}")
print(f"  - Priemerný 5-day return: {low_vol['target'].mean():.4f} ({low_vol['target'].mean()*100:.2f}%)")
print(f"  - Štandardná odchýlka: {low_vol['target'].std():.4f}")
print(f"  - Max return: {low_vol['target'].max():.4f}")
print(f"  - Min return: {low_vol['target'].min():.4f}")
print("="*60)

print("\n💡 Vysoká volatilita = väčšie príležitosti, ale aj väčšie riziká!")

---

## 💾 KROK 8: Ulož Dáta (Ako v Pipeline)

Uložíme train a test sety do CSV súborov.

In [ ]:
print("💾 Ukladám dáta...\n")

# Vytvor adresáre ak neexistujú
import os
os.makedirs('data/notebook_output', exist_ok=True)

# Ulož train set
train_df.to_csv('data/notebook_output/train_data.csv')
train_df[feature_cols].to_csv('data/notebook_output/train_X.csv')
train_df['target'].to_csv('data/notebook_output/train_y.csv')

# Ulož test set
test_df.to_csv('data/notebook_output/test_data.csv')
test_df[feature_cols].to_csv('data/notebook_output/test_X.csv')
test_df['target'].to_csv('data/notebook_output/test_y.csv')

print("✅ Dáta uložené v data/notebook_output/")
print("\nSúbory:")
print("  - train_data.csv, train_X.csv, train_y.csv")
print("  - test_data.csv, test_X.csv, test_y.csv")

---

## 🎉 HOTOVO!

### 🎓 Čo si sa naučil:

1. ✅ Ako stiahnuť finančné dáta z Yahoo Finance
2. ✅ Ako vytvoriť technické indikátory (returns, volatilita, trendy)
3. ✅ Ako vytvoriť target variable bez look-ahead bias
4. ✅ Ako správne rozdeliť time-series dáta (temporálne!)
5. ✅ Ako analyzovať koreláciu features s targetom
6. ✅ Ako vizualizovať finančné dáta

### 🚀 Ďalšie kroky:

1. **Experimentuj s rôznymi tickermi** - Zmeň `TICKER` na začiatku notebooku
2. **Skús rôzne holding periods** - Zmeň `HOLDING_PERIOD`
3. **Vytvor vlastné features** - Pridaj nové indikátory do `FeatureEngineer`
4. **Začni s ML modelmi** - Ridge Regression, XGBoost (Phase 2)

---

### 💡 Užitočné tipy:

- Spusti celý notebook: `Cell → Run All`
- Zmeň parametre na začiatku a spusti znova
- Skús rôzne vizualizácie - matplotlib a seaborn sú veľmi silné!

---

**Have fun exploring! 🎮📊**